In [1]:
import os
import scipy.io as sio
import scipy.misc as smisc
import numpy as np
import re as regexp
import math
import matplotlib.pyplot as plt
import scipy
import glob,os
import re

In [2]:
TotalDose = [50,100,150,200,250] # all the total dose that will be used
FrameDose = [0.5,1,2,4,6,8,10,12] # all the frame dose that will be used
path = '/srv/home/chenyu/NRR/STO_NRR/QuantTest/'

In [3]:
# make folders
base = '/srv/home/chenyu/NRR/STO_NRR/QuantTest/'
for i1 in range(len(TotalDose)):
    path = str(TotalDose[i1]) + '/'
    if os.path.exists(base + path) == False:
        os.mkdir(base + path)
    for i2 in range(len(FrameDose)):
        path = str(TotalDose[i1]) + '/' + str(FrameDose[i2]) + '/'
        if os.path.exists(base + path) == False:
            os.mkdir(base + path)

In [4]:
# Create parameter file for single batch
default_file = '/srv/home/chenyu/NRR/STO_NRR/DoseTest/NRR_B1.par'
with open(default_file) as f:
    datafile = f.readlines()
    for i1 in range(len(TotalDose)):
        for i2 in range(len(FrameDose)):
            path = str(TotalDose[i1]) + '/' + str(FrameDose[i2]) + '/'
            par_file = base + path + 'NRR_B1.par'
            print(par_file)
            f2 = open(par_file,"w+")
            for i in range(len(datafile)):
#                         print(datafile[i])
                temp = datafile[i].split(' ')
                if temp[0] == 'templateNamePattern':
                    f2.write('templateNamePattern /srv/home/chenyu/NRR/STO_NRR/QuantTest/' + str(FrameDose[i2]) +\
                             'us/STO_%03d_TEM.q2\n')
                elif temp[0] == 'numTemplates':
                    f2.write('numTemplates ' + str(int(TotalDose[i1]/FrameDose[i2])) + '\n')
                elif temp[0] == 'saveDirectory':
                    f2.write('saveDirectory '+ path + 'HAADF_NRR/')
                else:
                    f2.write(datafile[i])
            f2.close()

/srv/home/chenyu/NRR/STO_NRR/QuantTest/50/0.5/NRR_B1.par
/srv/home/chenyu/NRR/STO_NRR/QuantTest/50/1/NRR_B1.par
/srv/home/chenyu/NRR/STO_NRR/QuantTest/50/2/NRR_B1.par
/srv/home/chenyu/NRR/STO_NRR/QuantTest/50/4/NRR_B1.par
/srv/home/chenyu/NRR/STO_NRR/QuantTest/50/6/NRR_B1.par
/srv/home/chenyu/NRR/STO_NRR/QuantTest/50/8/NRR_B1.par
/srv/home/chenyu/NRR/STO_NRR/QuantTest/50/10/NRR_B1.par
/srv/home/chenyu/NRR/STO_NRR/QuantTest/50/12/NRR_B1.par
/srv/home/chenyu/NRR/STO_NRR/QuantTest/100/0.5/NRR_B1.par
/srv/home/chenyu/NRR/STO_NRR/QuantTest/100/1/NRR_B1.par
/srv/home/chenyu/NRR/STO_NRR/QuantTest/100/2/NRR_B1.par
/srv/home/chenyu/NRR/STO_NRR/QuantTest/100/4/NRR_B1.par
/srv/home/chenyu/NRR/STO_NRR/QuantTest/100/6/NRR_B1.par
/srv/home/chenyu/NRR/STO_NRR/QuantTest/100/8/NRR_B1.par
/srv/home/chenyu/NRR/STO_NRR/QuantTest/100/10/NRR_B1.par
/srv/home/chenyu/NRR/STO_NRR/QuantTest/100/12/NRR_B1.par
/srv/home/chenyu/NRR/STO_NRR/QuantTest/150/0.5/NRR_B1.par
/srv/home/chenyu/NRR/STO_NRR/QuantTest/150/1/N

In [5]:
# Create job submission file
# The only thing that needs to be changed on submission file is the job name
default_file = '/srv/home/chenyu/NRR/STO_NRR/QuantTest/NRR.sh'
njobs_per_submission = 4
ifile = 0
isubmission = 0
with open(default_file) as f:
    datafile = f.readlines()
    for i1 in range(len(TotalDose)):
        for i2 in range(len(FrameDose)):
            path = str(TotalDose[i1]) + '/' + str(FrameDose[i2]) + '/'
#             par_file = base + path + 'NRR.sh'
#             print(par_file)
#             f2 = open(par_file,"w+")
            if ifile == 0:
                submission_file = base + 'Submission_' + str(isubmission) + '.sh'
                print(submission_file)
                f2 = open(submission_file,"w+")
                f2.write('#!/bin/sh\n\n')
                f2.write('#SBATCH --job-name=Test_' + str(isubmission) + '\n')
                f2.write('#SBATCH --error=Test_' + str(isubmission) + '.err\n')
                f2.write('#SBATCH --output=Test_' + str(isubmission) + '.out\n')
                f2.write('#SBATCH --time=7-00:00:00\n')
                f2.write('#SBATCH --nodes=1\n')
                f2.write('#SBATCH --ntasks=2\n')
                f2.write('##SBATCH --export=ALL\n')
                f2.write('echo "Started on:"\ndate\n')
                f2.write('export LD_LIBRARY_PATH="/home/czhang376/bin/OpenBLAS/lib"\n\n')
                isubmission = isubmission + 1
            f2.write('/srv/home/chenyu/bin/NRR/quocGCC/projects/electronMicroscopy/matchSeries ' + \
                     path + 'NRR_B1.par\n')
            f2.write('rm -r ' + path + 'HAADF_NRR/stage1 ' + path + 'HAADF_NRR/stage2 ' + \
                    path + 'HAADF_NRR/stage3/*-r\n\n')
            ifile = ifile + 1
            
            if ifile == njobs_per_submission:
                ifile = 0
                f2.write('echo "Finished on:"\ndate\n')
                f2.close()
                # After the whole file writing, replace Windows style \r\n with Unix style \n
                fileContents = open(submission_file,"r").read()
                f2 = open(submission_file,"w", newline="\n")
                f2.write(fileContents)
                f2.close()

/srv/home/chenyu/NRR/STO_NRR/QuantTest/Submission_0.sh
/srv/home/chenyu/NRR/STO_NRR/QuantTest/Submission_1.sh
/srv/home/chenyu/NRR/STO_NRR/QuantTest/Submission_2.sh
/srv/home/chenyu/NRR/STO_NRR/QuantTest/Submission_3.sh
/srv/home/chenyu/NRR/STO_NRR/QuantTest/Submission_4.sh
/srv/home/chenyu/NRR/STO_NRR/QuantTest/Submission_5.sh
/srv/home/chenyu/NRR/STO_NRR/QuantTest/Submission_6.sh
/srv/home/chenyu/NRR/STO_NRR/QuantTest/Submission_7.sh
/srv/home/chenyu/NRR/STO_NRR/QuantTest/Submission_8.sh
/srv/home/chenyu/NRR/STO_NRR/QuantTest/Submission_9.sh
